In [1]:
from data_collector import load_model, load_data, preprocess_data,make_predictions, prepare_testing_data, save_model, split_data, create_input_target, reshape_input, create_model
from utils import clear_and_show_title, ensure_folders_exist
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import os

In [2]:
# If in src folder, go back to root folder
if os.getcwd().split('/')[-1] == 'src':
    os.chdir('..')
    

In [3]:
clear_and_show_title()
validation_size = 0.2
placeholder_symbol = "MAT"
train_n_days = 90
model = load_model()
data = load_data(stock="MAT")
# NOTE: Scaled all data; we will save for rescaling later
scaler, scaled_data = preprocess_data(data)
scaled_train_data, scaled_test_data = split_data(scaled_data)
print(scaled_train_data.shape, scaled_test_data.shape, scaled_data.shape)
print(scaled_train_data.shape[0] + scaled_test_data.shape[0] == scaled_data.shape[0])

||          WELCOME TO            ||
||        CRYPTO-ANALYST          ||
||         VERSION 1.0.0          ||
(9496, 1) (2375, 1) (11871, 1)
True


In [6]:
model_inputs = scaled_data[len(scaled_data) - len(scaled_test_data) - train_n_days:].values
model_inputs = model_inputs.reshae(-1, 1)
print(model_inputs.shape)

x_test = []
# 
for x in range(train_n_days, len(model_inputs)):
    x_test.append(model_inputs[x - train_n_days:x, 0])

print(x_test)
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

prediction_prices = make_predictions(model, x_test, scaler)
print(prediction_prices)


AttributeError: 'numpy.ndarray' object has no attribute 'reshae'

In [7]:

if model is None:
    x_train, y_train = create_input_target(train_data, train_n_days)
    x_train = reshape_input(x_train)
    # NOTE: We will use the last 20% of the training data as validation data
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=validation_size, shuffle=False)
    model, _ = create_model(x_train, y_train, x_val, y_val)  # Modify this line
    save_model(model)
else:
    print("[INFO]\tModel loaded from storage.")


predicted_prices = make_predictions(model, x_test, scaler)
# Create a dataframe containing two columns. One column with price and the other column stating whether the price is actual or predicted then merge into a single dataframe
# Filter test data to last n days
test_data = test_data[-90:]
# Create a new column indicating whether the price is actual or predicted
test_data['Type'] = 'Actual'

# Get latest date from test data
today = test_data['Date'].iloc[-1]
# get the next day in dd/mm/yyyy format
today = datetime.strptime(today, '%d/%m/%Y') + timedelta(days=1)
future_day = today.strftime('%d/%m/%Y')

# Add predicted prices as new row to test data for next day (i.e. the day after the last day of the test data)
test_data = test_data.append({'Date': future_day, 'Close': predicted_prices[0][0], 'Type': 'Predicted'}, ignore_index=True)
test_data.to_csv("results/test_data.csv", index=False)
print(test_data.tail())

[INFO]	Model loaded from storage.
75/75 [==============================] - 1s 17ms/step


NameError: name 'test_data' is not defined